In [86]:
# # Probably more imports than are really necessary...
# import os
# import torch
# import torchaudio
# from torch.utils.data import Dataset, DataLoader, random_split
# import torch.nn as nn
# import torch.nn.functional as F
# from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
# from tqdm import tqdm
# import librosa
# import numpy as np
# import miditoolkit
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import f1_score, average_precision_score, accuracy_score
# import random

## Metrics

In [87]:
def accuracy1(groundtruth, predictions):
    correct = 0
    for k in groundtruth:
        if not (k in predictions):
            print("Missing " + str(k) + " from predictions")
            return 0
        if predictions[k] == groundtruth[k]:
            correct += 1
    return correct / len(groundtruth)

In [88]:
def accuracy2(groundtruth, predictions):
    correct = 0
    for k in groundtruth:
        if not (k in predictions):
            print("Missing " + str(k) + " from predictions")
            return 0
        if predictions[k] == groundtruth[k]:
            correct += 1
    return correct / len(groundtruth)

In [89]:
TAGS = ['rock', 'oldies', 'jazz', 'pop', 'dance',  'blues',  'punk', 'chill', 'electronic', 'country']

In [90]:
def accuracy3(groundtruth, predictions):
    preds, targets = [], []
    for k in groundtruth:
        if not (k in predictions):
            print("Missing " + str(k) + " from predictions")
            return 0
        prediction = [1 if tag in predictions[k] else 0 for tag in TAGS]
        target = [1 if tag in groundtruth[k] else 0 for tag in TAGS]
        preds.append(prediction)
        targets.append(target)
    
    mAP = average_precision_score(targets, preds, average='macro')
    return mAP

## Task 1: Composer classification

In [91]:
dataroot1 = "student_files/task1_composer_classification"

In [92]:
# import os
# import miditoolkit
# import numpy as np
# from lightgbm import LGBMClassifier
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from sklearn.utils import resample
# from music21 import converter, chord
# from collections import Counter

# class model1():
#     def __init__(self):
#         self.model = None
#         self.label_encoder = LabelEncoder()
#         self.chord_vocab = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B", "N"]

#     def extract_features(self, path):
#         midi_path = os.path.join(dataroot1, path)
#         try:
#             midi = miditoolkit.MidiFile(midi_path)
#             notes = [note for inst in midi.instruments for note in inst.notes]
#         except:
#             return [0.0] * (9 + len(self.chord_vocab))

#         if not notes:
#             return [0.0] * (9 + len(self.chord_vocab))

#         pitches = [note.pitch for note in notes]
#         durations = [note.end - note.start for note in notes]
#         velocities = [note.velocity for note in notes]

#         pitch_range = max(pitches) - min(pitches)
#         density = len(notes) / (midi.max_tick + 1)

#         try:
#             score = converter.parse(midi_path)
#             key_sig = hash(str(score.analyze('key'))) % 1000
#             chords = score.chordify().recurse().getElementsByClass(chord.Chord)
#             chord_roots = [c.root().name if c.isTriad() else "N" for c in chords]
#             chord_count = Counter(chord_roots)
#             chord_vector = [int(chord_count.get(name, 0) > 0) for name in self.chord_vocab]
#         except:
#             key_sig = 0
#             chord_vector = [0] * len(self.chord_vocab)

#         base_features = [
#             np.mean(pitches),
#             np.std(pitches),
#             np.min(pitches),
#             np.max(pitches),
#             pitch_range,
#             np.mean(durations),
#             np.std(durations),
#             np.mean(velocities),
#             len(notes),
#             density,
#             key_sig
#         ]

#         return base_features + chord_vector

#     def train(self, path):
#         data = eval(open(path).read())
#         X_raw = [self.extract_features(k) for k in data]
#         y_raw = [data[k] for k in data]
#         self.label_encoder.fit(y_raw)
#         y_encoded = self.label_encoder.transform(y_raw)

#         # Split into train/val (50% split)
#         X_train, X_val, y_train, y_val = train_test_split(
#             X_raw, y_encoded, test_size=0.5, stratify=y_encoded, random_state=42)

#         # Upsample training set
#         X_up, y_up = [], []
#         class_labels = np.unique(y_train)
#         max_count = max([np.sum(np.array(y_train) == cls) for cls in class_labels])

#         for cls in class_labels:
#             X_cls = [x for x, y in zip(X_train, y_train) if y == cls]
#             y_cls = [y for y in y_train if y == cls]
#             X_res, y_res = resample(X_cls, y_cls, replace=True, n_samples=max_count, random_state=42)
#             X_up.extend(X_res)
#             y_up.extend(y_res)

#         print("✅ Upsampled training set to", len(X_up), "samples")

#         self.model = LGBMClassifier(max_depth=5, learning_rate=0.05, n_estimators=200)
#         self.model.fit(X_up, y_up)

#         val_acc = self.model.score(X_val, y_val)
#         print("✅ Validation accuracy (50% holdout):", val_acc)

#     def predict(self, path, outpath=None):
#         data = eval(open(path).read())
#         X = [self.extract_features(k) for k in data]
#         preds = self.model.predict(X)
#         preds_decoded = self.label_encoder.inverse_transform(preds)
#         predictions = {k: p for k, p in zip(data, preds_decoded)}

#         if outpath:
#             with open(outpath, "w") as f:
#                 f.write(str(predictions) + "\n")

#         return predictions

In [93]:
# import os
# import miditoolkit
# import numpy as np
# from xgboost import XGBClassifier
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import GridSearchCV
# from music21 import converter, chord
# from collections import Counter

# class model1():
#     def __init__(self):
#         self.model = None
#         self.label_encoder = LabelEncoder()
#         self.chord_vocab = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B", "N"]

#     def extract_features(self, path):
#         midi_path = os.path.join(dataroot1, path)
#         try:
#             midi = miditoolkit.MidiFile(midi_path)
#             notes = [note for inst in midi.instruments for note in inst.notes]
#         except:
#             return [0.0] * (11 + len(self.chord_vocab))

#         if not notes:
#             return [0.0] * (11 + len(self.chord_vocab))

#         pitches = [note.pitch for note in notes]
#         durations = [note.end - note.start for note in notes]
#         velocities = [note.velocity for note in notes]

#         pitch_range = max(pitches) - min(pitches)
#         density = len(notes) / (midi.max_tick + 1)

#         try:
#             score = converter.parse(midi_path)
#             key_sig = hash(str(score.analyze('key'))) % 1000
#             chords = score.chordify().recurse().getElementsByClass(chord.Chord)
#             chord_roots = [c.root().name if c.isTriad() else "N" for c in chords]
#             chord_count = Counter(chord_roots)
#             chord_vector = [chord_count.get(name, 0) for name in self.chord_vocab]
#         except:
#             key_sig = 0
#             chord_vector = [0] * len(self.chord_vocab)

#         base_features = [
#             np.mean(pitches),
#             np.std(pitches),
#             np.min(pitches),
#             np.max(pitches),
#             pitch_range,
#             np.mean(durations),
#             np.std(durations),
#             np.mean(velocities),
#             len(notes),
#             density,
#             key_sig
#         ]

#         return base_features + chord_vector

#     def train(self, path):
#         data = eval(open(path).read())
#         X = [self.extract_features(k) for k in data]
#         y = [data[k] for k in data]
#         self.label_encoder.fit(y)
#         y_encoded = self.label_encoder.transform(y)

#         param_grid = {
#             'max_depth': [5, 6],
#             'learning_rate': [0.05],
#             'n_estimators': [200]
#         }

#         grid = GridSearchCV(
#             XGBClassifier(eval_metric='mlogloss'),
#             param_grid,
#             scoring='accuracy',
#             cv=3,
#             n_jobs=-1
#         )

#         grid.fit(X, y_encoded)
#         self.model = grid.best_estimator_
#         print("✅ GridSearch best params:", self.model.get_params())
#         print("✅ Training accuracy:", self.model.score(X, y_encoded))

#     def predict(self, path, outpath=None):
#         data = eval(open(path).read())
#         X = [self.extract_features(k) for k in data]
#         preds = self.model.predict(X)
#         preds_decoded = self.label_encoder.inverse_transform(preds)
#         predictions = {k: p for k, p in zip(data, preds_decoded)}

#         if outpath:
#             with open(outpath, "w") as f:
#                 f.write(str(predictions) + "\n")

#         return predictions


In [94]:
# ✅ GridSearch best params: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.05, 'max_depth': 7, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 200, 'n_jobs': None, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}

## Task 2: Sequence prediction

In [95]:
dataroot2 = "student_files/student_files/task2_next_sequence_prediction"

In [96]:
# import os
# import miditoolkit
# import numpy as np
# from xgboost import XGBClassifier
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import GridSearchCV, StratifiedKFold
# from sklearn.feature_selection import SelectKBest, f_classif, RFECV
# from music21 import converter

# class model2():
#     def __init__(self):
#         self.selector = None
#         self.model = None

#     def extract_features(self, path):
#         path = os.path.join(dataroot2, path)
#         try:
#             midi = miditoolkit.MidiFile(path)
#             notes = [note for inst in midi.instruments for note in inst.notes]
#         except:
#             return [0.0] * 8, set(), 120, 0.0, 'unknown', 'unknown'

#         if not notes:
#             return [0.0] * 8, set(), 120, 0.0, 'unknown', 'unknown'

#         pitches = [note.pitch for note in notes]
#         durations = [note.end - note.start for note in notes]
#         velocities = [note.velocity for note in notes]
#         pitch_range = max(pitches) - min(pitches)
#         density = len(notes) / (midi.max_tick + 1)

#         try:
#             score = converter.parse(path)
#             key_sig = str(score.analyze('key'))
#             time_sig = str(score.getTimeSignatures()[0].ratioString)
#         except:
#             key_sig = 'unknown'
#             time_sig = 'unknown'

#         return [
#             np.mean(pitches),
#             np.min(pitches),
#             np.max(pitches),
#             pitch_range,
#             np.mean(durations),
#             np.mean(velocities),
#             len(notes),
#             density
#         ], set(pitches), 120, density, key_sig, time_sig

#     def combine_features(self, f1, f2):
#         x1, pitch_set1, tempo1, density1, key1, time1 = self.extract_features(f1)
#         x2, pitch_set2, tempo2, density2, key2, time2 = self.extract_features(f2)
#         diff = np.abs(np.array(x1) - np.array(x2)).tolist()
#         sqdiff = ((np.array(x1) - np.array(x2)) ** 2).tolist()
#         overlap = len(pitch_set1 & pitch_set2) / max(len(pitch_set1 | pitch_set2), 1)
#         density_diff = abs(density1 - density2)
#         key_match = int(key1 == key2)
#         time_match = int(time1 == time2)
#         return x1 + x2 + diff + sqdiff + [overlap, density_diff, key_match, time_match]

#     def train(self, path):
#         data = eval(open(path).read())
#         X = [self.combine_features(f1, f2) for (f1, f2) in data]
#         y = [int(data[(f1, f2)]) for (f1, f2) in data]

#         param_grid = {
#             'max_depth': [6, 7],
#             'learning_rate': [0.05],
#             'n_estimators': [200]
#         }

#         base_model = XGBClassifier(eval_metric='logloss')
#         grid = GridSearchCV(base_model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
#         grid.fit(X, y)
#         best_model = grid.best_estimator_

#         print("✅ GridSearch best params:", best_model.get_params())

#         rfecv = RFECV(
#             estimator=best_model,
#             step=1,
#             min_features_to_select=5,
#             cv=StratifiedKFold(3),
#             scoring='accuracy',
#             n_jobs=-1,
#             verbose=1
#         )

#         rfecv.fit(X, y)
#         self.model = rfecv.estimator_
#         self.selector = rfecv

#         print("✅ RFECV selected", rfecv.n_features_, "features.")
#         print("✅ Training accuracy:", rfecv.score(X, y))

#     def predict(self, path, outpath=None):
#         data = eval(open(path).read())
#         X = [self.combine_features(f1, f2) for (f1, f2) in data]
#         X_selected = self.selector.transform(X)

#         preds = self.model.predict(X_selected)
#         keys = list(data)
#         predictions = {k: bool(p) for k, p in zip(keys, preds)}

#         if outpath:
#             with open(outpath, 'w') as f:
#                 f.write(str(predictions) + "\n")
#         return predictions


## Task 3: Audio classification

In [97]:
root = "student_files/task3_audio_classification"

In [98]:
import os
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.model_selection import train_test_split
import librosa
import numpy as np
from tqdm import tqdm

TAGS = ['rock', 'oldies', 'jazz', 'pop', 'dance', 'blues', 'punk', 'chill', 'electronic', 'country']
SAMPLE_RATE = 16000
N_MELS = 64
N_CLASSES = len(TAGS)
AUDIO_DURATION = 10  # seconds
BATCH_SIZE = 32
LABEL_SMOOTHING = 0.1

class AudioDataset(Dataset):
    def __init__(self, meta, root, train=False, preload=False):
        self.meta = meta
        self.root = root
        self.train = train
        self.paths = list(meta.keys())
        self.labels = [meta[k] for k in self.paths]
        self.mel = MelSpectrogram(sample_rate=SAMPLE_RATE, n_mels=N_MELS)
        self.db = AmplitudeToDB()
        self.preload = preload
        self.pathToFeat = {}

        if preload:
            for path in tqdm(self.paths, desc="Preloading features"):
                full_path = os.path.join(self.root, path)
                waveform, sr = librosa.load(full_path, sr=SAMPLE_RATE)
                waveform = np.array([waveform])
                pad_len = max(0, SAMPLE_RATE * AUDIO_DURATION - waveform.shape[1])
                if pad_len > 0:
                    waveform = F.pad(torch.tensor(waveform), (0, pad_len))
                else:
                    waveform = torch.tensor(waveform[:, :SAMPLE_RATE * AUDIO_DURATION])
                mel_spec = self.db(self.mel(waveform)).squeeze(0)
                self.pathToFeat[path] = mel_spec

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        if self.preload:
            mel_spec = self.pathToFeat[path]
        else:
            full_path = os.path.join(self.root, path)
            waveform, sr = librosa.load(full_path, sr=SAMPLE_RATE)
            waveform = np.array([waveform])
            pad_len = max(0, SAMPLE_RATE * AUDIO_DURATION - waveform.shape[1])
            if pad_len > 0:
                waveform = F.pad(torch.tensor(waveform), (0, pad_len))
            else:
                waveform = torch.tensor(waveform[:, :SAMPLE_RATE * AUDIO_DURATION])
            mel_spec = self.db(self.mel(waveform)).squeeze(0)

        if self.train:
            if torch.rand(1).item() < 0.5:
                mel_spec = mel_spec + 0.01 * torch.randn_like(mel_spec)
            if torch.rand(1).item() < 0.5:
                mel_spec = torch.roll(mel_spec, shifts=np.random.randint(-10, 10), dims=-1)

        multi_hot = torch.tensor([1 if tag in self.meta[path] else 0 for tag in TAGS], dtype=torch.float32)
        smoothed = (1 - LABEL_SMOOTHING) * multi_hot + LABEL_SMOOTHING / N_CLASSES
        return mel_spec.unsqueeze(0), smoothed, path

class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Sequential(
            nn.Linear(64 * (N_MELS // 8) * (801 // 8), 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, N_CLASSES))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class Pipeline:
    def __init__(self, model, lr=1e-4):
        self.model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=15)
        self.device = next(model.parameters()).device

    def train(self, loader_train, loader_val, num_epochs):
        for epoch in range(num_epochs):
            self.model.train()
            total_loss = 0
            for x, y, _ in tqdm(loader_train, desc=f"Epoch {epoch+1}"):
                x, y = x.to(self.device), y.to(self.device)
                out = self.model(x)
                loss = self.criterion(out, y)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
            self.scheduler.step()
            print(f"[Epoch {epoch+1}] Loss: {total_loss / len(loader_train):.4f}")

    def evaluate(self, loader, threshold=0.5, outpath=None):
        self.model.eval()
        predictions = {}
        targets = []
        preds_raw = []
        paths = []
        with torch.no_grad():
            for x, y, ps in loader:
                x = x.to(self.device)
                out = self.model(x).cpu()
                preds_raw.append(out)
                targets.append(y)
                paths.extend(ps)

        preds_raw = torch.cat(preds_raw)
        targets = torch.cat(targets)

        thresholds = []
        for i in range(N_CLASSES):
            precision, recall, thresh = precision_recall_curve(targets[:, i], preds_raw[:, i])
            f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
            best_thresh = thresh[f1[:-1].argmax()] if len(thresh) > 0 else 0.5
            thresholds.append(best_thresh)

        for i in range(len(paths)):
            pred_vec = (preds_raw[i] > torch.tensor(thresholds).to(preds_raw.device)).float()
            predictions[paths[i]] = [TAGS[j] for j in range(N_CLASSES) if pred_vec[j] > 0.5]

        mAP = average_precision_score(targets, preds_raw, average='macro')
        print("Validation mAP:", mAP)

        if outpath:
            with open(outpath, 'w') as f:
                f.write(str(predictions) + "\n")
        else:
            return predictions, mAP


In [107]:
import random

In [ ]:
class Loaders():
    def __init__(self, train_path, test_path, split_ratio=0.8, seed=0):
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

        meta_train = eval(open(train_path, 'r').read())
        l_test = eval(open(test_path, 'r').read())
        meta_test = {x: [] for x in l_test}

        full_train = list(meta_train.items())
        random.shuffle(full_train)
        meta_train = dict(full_train)

        all_train = AudioDataset(meta_train, root, train=True, preload=True)
        test_set = AudioDataset(meta_test, root, preload=False)

        # Split into train + valid
        total_len = len(all_train)
        train_len = int(total_len * split_ratio)
        valid_len = total_len - train_len
        train_set, valid_set = torch.utils.data.random_split(all_train, [train_len, valid_len])

        self.loaderTrain = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
        self.loaderValid = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderTest  = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [100]:
# ohh ok let's try this. This code is over the baseline on validation set, so i want you to use this to run prediction on test set: import os
# import torch
# import torchaudio
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
# from sklearn.metrics import average_precision_score
# import librosa
# import numpy as np
# from tqdm import tqdm

# TAGS = ['rock', 'oldies', 'jazz', 'pop', 'dance', 'blues', 'punk', 'chill', 'electronic', 'country']
# SAMPLE_RATE = 16000
# N_MELS = 64
# N_CLASSES = len(TAGS)
# AUDIO_DURATION = 10  # seconds
# BATCH_SIZE = 32

# # --- Dataset with optional Preload and Augmentation ---
# class AudioDataset(Dataset):
#     def __init__(self, meta, root, train=False, preload=False):
#         self.meta = meta
#         self.root = root
#         self.train = train
#         self.paths = list(meta.keys())
#         self.labels = [meta[k] for k in self.paths]
#         self.mel = MelSpectrogram(sample_rate=SAMPLE_RATE, n_mels=N_MELS)
#         self.db = AmplitudeToDB()
#         self.preload = preload
#         self.pathToFeat = {}

#         if preload:
#             for path in tqdm(self.paths, desc="Preloading features"):
#                 full_path = os.path.join(self.root, path)
#                 waveform, sr = librosa.load(full_path, sr=SAMPLE_RATE)
#                 waveform = np.array([waveform])
#                 pad_len = max(0, SAMPLE_RATE * AUDIO_DURATION - waveform.shape[1])
#                 if pad_len > 0:
#                     waveform = F.pad(torch.tensor(waveform), (0, pad_len))
#                 else:
#                     waveform = torch.tensor(waveform[:, :SAMPLE_RATE * AUDIO_DURATION])
#                 mel_spec = self.db(self.mel(waveform)).squeeze(0)
#                 self.pathToFeat[path] = mel_spec

#     def __len__(self):
#         return len(self.paths)

#     def __getitem__(self, idx):
#         path = self.paths[idx]
#         if self.preload:
#             mel_spec = self.pathToFeat[path]
#         else:
#             full_path = os.path.join(self.root, path)
#             waveform, sr = librosa.load(full_path, sr=SAMPLE_RATE)
#             waveform = np.array([waveform])
#             pad_len = max(0, SAMPLE_RATE * AUDIO_DURATION - waveform.shape[1])
#             if pad_len > 0:
#                 waveform = F.pad(torch.tensor(waveform), (0, pad_len))
#             else:
#                 waveform = torch.tensor(waveform[:, :SAMPLE_RATE * AUDIO_DURATION])
#             mel_spec = self.db(self.mel(waveform)).squeeze(0)

#         if self.train:
#             if torch.rand(1).item() < 0.5:
#                 mel_spec = mel_spec + 0.01 * torch.randn_like(mel_spec)
#             if torch.rand(1).item() < 0.5:
#                 mel_spec = torch.roll(mel_spec, shifts=np.random.randint(-10, 10), dims=-1)

#         multi_hot = torch.tensor([1 if tag in self.labels[idx] else 0 for tag in TAGS], dtype=torch.float32)
#         return mel_spec.unsqueeze(0), multi_hot, path

# # --- Loaders ---
# class Loaders:
#     def __init__(self, train_path, test_path, root):
#         import random
#         torch.manual_seed(0)
#         random.seed(0)

#         meta_train = eval(open(train_path, 'r').read())
#         test_files = eval(open(test_path, 'r').read())
#         meta_test = {x: [] for x in test_files}

#         full_train = list(meta_train.items())
#         split = int(0.9 * len(full_train))
#         random.shuffle(full_train)
#         meta_train, meta_valid = dict(full_train[:split]), dict(full_train[split:])

#         self.loaderTrain = DataLoader(AudioDataset(meta_train, root, train=True, preload=True), batch_size=BATCH_SIZE, shuffle=True)
#         self.loaderValid = DataLoader(AudioDataset(meta_valid, root, preload=True), batch_size=BATCH_SIZE)
#         self.loaderTest = DataLoader(AudioDataset(meta_test, root, preload=False), batch_size=BATCH_SIZE)

# # --- CNN Model ---
# class CNNClassifier(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(1, 16, 3, padding=1),
#             nn.BatchNorm2d(16),
#             nn.ReLU(),
#             nn.MaxPool2d(2))
#         self.conv2 = nn.Sequential(
#             nn.Conv2d(16, 32, 3, padding=1),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(2))
#         self.conv3 = nn.Sequential(
#             nn.Conv2d(32, 64, 3, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2))
#         self.fc = nn.Sequential(
#             nn.Linear(64 * (N_MELS // 8) * (801 // 8), 256),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(256, N_CLASSES))

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.conv3(x)
#         x = x.view(x.size(0), -1)
#         return self.fc(x)

# # --- Training and Evaluation ---
# class Pipeline:
#     def __init__(self, model, lr=1e-4):
#         self.model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
#         self.criterion = nn.BCEWithLogitsLoss()
#         self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
#         self.device = next(model.parameters()).device

#     def train(self, loader, num_epochs):
#         for epoch in range(num_epochs):
#             self.model.train()
#             total_loss = 0
#             for x, y, _ in tqdm(loader, desc=f"Epoch {epoch+1}"):
#                 x, y = x.to(self.device), y.to(self.device)
#                 out = self.model(x)
#                 loss = self.criterion(out, y)
#                 self.optimizer.zero_grad()
#                 loss.backward()
#                 self.optimizer.step()
#                 total_loss += loss.item()
#             print(f"[Epoch {epoch+1}] Loss: {total_loss / len(loader):.4f}")

#     def evaluate(self, loader, outpath=None, top_k=2):
#         self.model.eval()
#         predictions = {}
#         targets = []
#         preds_raw = []
#         paths = []
#         with torch.no_grad():
#             for x, y, ps in loader:
#                 x = x.to(self.device)
#                 out = self.model(x).cpu()
#                 preds_raw.append(out)
#                 targets.append(y)
#                 paths.extend(ps)

#         preds_raw = torch.cat(preds_raw)
#         targets = torch.cat(targets)

#         for i in range(len(paths)):
#             topk = preds_raw[i].topk(top_k).indices
#             predictions[paths[i]] = [TAGS[j] for j in topk]

#         if outpath:
#             with open(outpath, 'w') as f:
#                 f.write(str(predictions) + "\n")
#         else:
#             mAP = average_precision_score(targets, preds_raw, average='macro')
#             print("Validation mAP:", mAP)
#             return predictions, mAP

## Run everything...

In [101]:
# def run1():
#     model = model1()
#     model.train(dataroot1 + "/train.json")
#     train_preds = model.predict(dataroot1 + "/train.json")
#     test_preds = model.predict(dataroot1 + "/test.json", "predictions1.json")
#     train_labels = eval(open(dataroot1 + "/train.json").read())
#     acc1 = accuracy1(train_labels, train_preds)
#     print("Task 1 training accuracy = " + str(acc1))

In [102]:
# def run2():
#     model = model2()
#     model.train(dataroot2 + "/train.json")
#     train_preds = model.predict(dataroot2 + "/train.json")
#     test_preds = model.predict(dataroot2 + "/test.json", "predictions2.json")
    
#     train_labels = eval(open(dataroot2 + "/train.json").read())
#     acc2 = accuracy2(train_labels, train_preds)
#     print("Task 2 training accuracy = " + str(acc2))

In [103]:
def run3():
    loaders = Loaders(root + "/train.json", root + "/test.json")
    model = CNNClassifier()
    pipeline = Pipeline(model, 1e-4)
    
    pipeline.train(loaders.loaderTrain, loaders.loaderValid, 5)
    train_preds, train_mAP = pipeline.evaluate(loaders.loaderTrain, 0.5)
    valid_preds, valid_mAP = pipeline.evaluate(loaders.loaderValid, 0.5)
    test_preds, _ = pipeline.evaluate(loaders.loaderTest, 0.5, "predictions3.json")
    
    all_train = eval(open(root + "/train.json").read())
    for k in valid_preds:
        # We split our training set into train+valid
        # so need to remove validation instances from the training set for evaluation
        all_train.pop(k)
    acc3 = accuracy3(all_train, train_preds)
    print("Task 3 training mAP = " + str(acc3))

In [104]:
# run1()

In [105]:
# run2()

In [106]:
run3()

NameError: name 'random' is not defined